# AD_Tech_SLM - DPO Training Notebook

このノートブックは、広告特化型SLMのDPOトレーニングを実行するためのインタラクティブな環境です。

## 環境
- MacBook Air M2 8GB
- Metal Performance Shaders (MPS) GPU
- DPO (Direct Preference Optimization) 手法

## 1. 環境設定

In [ ]:
import os
import sys
import torch
import pandas as pd
import json
from pathlib import Path

# プロジェクトルートを追加
project_root = Path().absolute().parent
sys.path.append(str(project_root))

print(f"Project root: {project_root}")
print(f"Current working directory: {os.getcwd()}")

In [ ]:
# デバイス確認
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("🚀 Metal Performance Shaders (MPS) is available!")
else:
    device = torch.device("cpu")
    print("⚠️ MPS not available, using CPU")

print(f"Using device: {device}")
print(f"PyTorch version: {torch.__version__}")

## 2. データセットの確認

In [ ]:
# データセットを読み込み
dataset_path = project_root / "data" / "sample_dpo_dataset.jsonl"

data = []
with open(dataset_path, 'r', encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line.strip()))

df = pd.DataFrame(data)
print(f"データセットのサンプル数: {len(df)}")
print(f"カラム: {list(df.columns)}")
df.head()

In [ ]:
# データの統計情報
print("=== データ統計 ===")
print(f"プロンプトの平均文字数: {df['prompt'].str.len().mean():.1f}")
print(f"chosenの平均文字数: {df['chosen'].str.len().mean():.1f}")
print(f"rejectedの平均文字数: {df['rejected'].str.len().mean():.1f}")

print("\n=== サンプル例 ===")
sample = df.iloc[0]
print(f"プロンプト: {sample['prompt']}")
print(f"Chosen: {sample['chosen']}")
print(f"Rejected: {sample['rejected']}")

## 3. モデルとトークナイザーの読み込み

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model, TaskType

# モデル名（M2 8GBに適したサイズ）
model_name = "google/gemma-2b-it"

print(f"モデルを読み込み中: {model_name}")

# トークナイザーの読み込み
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print("✅ トークナイザー読み込み完了")

In [ ]:
# モデルの読み込み
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto" if torch.backends.mps.is_available() else None,
)

print("✅ ベースモデル読み込み完了")
print(f"モデルパラメータ数: {sum(p.numel() for p in model.parameters()):,}")

## 4. LoRA設定

In [ ]:
# LoRA設定
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=16,  # LoRA rank
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],
)

# LoRAモデルの作成
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

print("✅ LoRA設定完了")

## 5. データ前処理

In [ ]:
from datasets import Dataset

# HuggingFace Datasetに変換
dataset = Dataset.from_pandas(df)

# 訓練・検証分割
train_size = int(len(dataset) * 0.8)
train_dataset = dataset.select(range(train_size))
eval_dataset = dataset.select(range(train_size, len(dataset)))

print(f"訓練データ: {len(train_dataset)} サンプル")
print(f"検証データ: {len(eval_dataset)} サンプル")

## 6. 簡単な推論テスト

In [ ]:
def generate_ad_copy(prompt, max_length=150):
    """広告コピーを生成する関数"""
    inputs = tokenizer.encode(prompt, return_tensors="pt")
    
    if torch.backends.mps.is_available():
        inputs = inputs.to("mps")
    
    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_length=max_length,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # プロンプト部分を除去
    if generated_text.startswith(prompt):
        generated_text = generated_text[len(prompt):].strip()
    
    return generated_text

# テスト実行
test_prompt = "【テーマ】雨の日でもワクワクするニュースアプリを紹介してください"
result = generate_ad_copy(test_prompt)

print(f"プロンプト: {test_prompt}")
print(f"生成結果: {result}")

## 7. DPO トレーニングの準備

実際のDPOトレーニングは `scripts/train_dpo.py` を使用して実行します。

ターミナルで以下のコマンドを実行してください：

```bash
cd /path/to/AD_Tech_SLM
python scripts/train_dpo.py
```

## 8. モデル評価

トレーニング後のモデルを評価するセクション

In [ ]:
# トレーニング済みモデルを読み込んで評価する場合
# このセクションはトレーニング完了後に実行

test_prompts = [
    "【テーマ】健康管理をサポートするフィットネスアプリを紹介してください",
    "【テーマ】料理初心者向けのレシピアプリを紹介してください",
    "【テーマ】読書好きのための電子書籍アプリを紹介してください",
]

print("=== 生成テスト ===")
for i, prompt in enumerate(test_prompts, 1):
    result = generate_ad_copy(prompt)
    print(f"\nテスト {i}:")
    print(f"プロンプト: {prompt}")
    print(f"生成結果: {result}")
    print("-" * 50)